Last Update:

- June 7, 2025
- ADK == 1.1.1

In this notebook:

- Simple request via `requests`.
- Advanced content retrieval via `selenium` and headless Chrome.
- Agent flow to handle multiple URL requests.

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.sandbox.google.com/github/hupili/google-adk-101/blob/main/Web_request_tool.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fhupili%2Fgoogle-adk-101%2Fmain%2FWeb_request_tool.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-enterprise-logo.png" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/hupili/google-adk-101/blob/main/Web_request_tool.ipynb">
      <img width="32px" src="https://www.gstatic.com/monospace/240802/git_host_github_mask.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/hupili/google-adk-101/main/Web_request_tool.ipynb">
      <img width="32px" src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Environment Setup

In [1]:
import os

# API_KEY = '' # @param {type:"string"}
# os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE" # Use Vertex AI API
# os.environ["GOOGLE_API_KEY"] = API_KEY

PROJECT_ID = "hupili-genai-bb"  # @param {type:"string"}
if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

STAGING_BUCKET = 'gs://agent_engine_deploy_staging' # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API

# [your-project-id]

In [2]:
import vertexai

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
!pip install deprecated
!pip install google-adk==1.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.5/269.5 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
  Created wheel for google-cloud-secret-manager: filename=google_cloud_secret_manager-2.24.0-py3-none-any.whl size=217964 sha256=b02f9a12c9ba1c1576b04d8172ad548c6

In [5]:
from google import adk

In [6]:
import json
import time

def pprint_events(events):
    '''Pretty print of events generated by ADK runner'''
    start_time = time.time()
    for _, event in enumerate(events):
        is_final_response = event.is_final_response()
        function_calls = event.get_function_calls()
        function_responses = event.get_function_responses()
        agent_res = json.loads(event.content.model_dump_json(indent=2, exclude_none=True))

        if is_final_response:
            print('>>> inside final response...')
            final_response = event.content.parts[0].text
            end_time = time.time()
            elapsed_time_ms = round((end_time - start_time) * 1000, 3)
            print(f'Final Response ({elapsed_time_ms} ms):\n{final_response}')
            print("-----------------------------")
        elif function_calls:
            print('+++ inside the function call...')
            for function_call in function_calls:
                print(f"function, [args]:  {function_call.name}, {function_call.args}")
        elif function_responses:
            print('--- inside the function call response...')
            # TODO(Pili): copied from walkthrough codes. Find root cause of 'pending' not found.
            # if not event.actions.pending:
            if True:
                for function_response in function_responses:
                    details = function_response.response
                    recommended_list = list(details.values())
                    print(f"Function Name: {function_response.name}")
                    result=json.dumps(recommended_list)
                    print(f"Function Results {result}")
            else:
                print(agent_res)
    print(f"Total elapsed time: {elapsed_time_ms}")


In [7]:
import random
import asyncio

from google.adk.tools import google_search
from google.adk import Agent
from google.adk.agents import Agent, LlmAgent
from google.genai import types
from pydantic import BaseModel
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

async def caller_factory(root_agent, app_name='App12345', user_id='User12345', session_id=None):
  session_service = InMemorySessionService()
  if session_id is None:
    suffix = random.randint(100000, 999999)
    session_id = f'{app_name}-{user_id}-{suffix}'
  session = await session_service.create_session(
      app_name=app_name, user_id=user_id, session_id=session_id)
  runner = Runner(agent=root_agent, app_name=app_name, session_service=session_service)
  def _call(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
    return events
  return _call

# Simple `requests`

In [15]:
MODEL = "gemini-2.0-flash"
APP_NAME = "web_tool_agent"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "web_tool_agent"

def request_url(url: str) -> str:
  '''Request an URL and return the HTML content as text.
  Args:
    url: The URL to request.
  Returns:
    The HTML content of the URL, or '' if the request fails.
  '''
  import requests
  try:
    response = requests.get(url)
    response.raise_for_status()
    return response.text
  except requests.exceptions.RequestException:
    return ''

# Agent
web_tool_agent = Agent(
    model=MODEL,
    name="web_tool_agent",
    description="Answer the user questions with web request tool",
    instruction="Answer the user questions as instructed. Use `request_url` tool if the question is related to web request.",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[request_url],
)

In [16]:
call_agent = await caller_factory(web_tool_agent)

In [17]:
events = call_agent('Hello')
pprint_events(events)

>>> inside final response...
Final Response (1177.052 ms):
Hello! How can I help you today?

-----------------------------
Total elapsed time: 1177.052


In [18]:
events = call_agent('what are the available notebooks in https://github.com/hupili/google-adk-101/tree/main ?')
pprint_events(events)

+++ inside the function call...
function, [args]:  request_url, {'url': 'https://github.com/hupili/google-adk-101/tree/main'}
--- inside the function call response...
Function Name: request_url
Function Results ["\n\n\n\n\n\n\n<!DOCTYPE html>\n<html\n  lang=\"en\"\n  \n  data-color-mode=\"auto\" data-light-theme=\"light\" data-dark-theme=\"dark\"\n  data-a11y-animated-images=\"system\" data-a11y-link-underlines=\"true\"\n  data-css-features=\"prs_diff_containment\"\n  >\n\n\n\n  <head>\n    <meta charset=\"utf-8\">\n  <link rel=\"dns-prefetch\" href=\"https://github.githubassets.com\">\n  <link rel=\"dns-prefetch\" href=\"https://avatars.githubusercontent.com\">\n  <link rel=\"dns-prefetch\" href=\"https://github-cloud.s3.amazonaws.com\">\n  <link rel=\"dns-prefetch\" href=\"https://user-images.githubusercontent.com/\">\n  <link rel=\"preconnect\" href=\"https://github.githubassets.com\" crossorigin>\n  <link rel=\"preconnect\" href=\"https://avatars.githubusercontent.com\">\n\n  \n\n 

In [19]:
events = call_agent('Get list of news title and their URL as markdown from https://news.ycombinator.com/')
pprint_events(events)

+++ inside the function call...
function, [args]:  request_url, {'url': 'https://news.ycombinator.com/'}
--- inside the function call response...
Function Name: request_url
Function Results ["<html lang=\"en\" op=\"news\"><head><meta name=\"referrer\" content=\"origin\"><meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\"><link rel=\"stylesheet\" type=\"text/css\" href=\"news.css?c5rFJeaXqANRegCfZJSx\">\n        <link rel=\"icon\" href=\"y18.svg\">\n                  <link rel=\"alternate\" type=\"application/rss+xml\" title=\"RSS\" href=\"rss\">\n        <title>Hacker News</title></head><body><center><table id=\"hnmain\" border=\"0\" cellpadding=\"0\" cellspacing=\"0\" width=\"85%\" bgcolor=\"#f6f6ef\">\n        <tr><td bgcolor=\"#ff6600\"><table border=\"0\" cellpadding=\"0\" cellspacing=\"0\" width=\"100%\" style=\"padding:2px\"><tr><td style=\"width:18px;padding-right:4px\"><a href=\"https://news.ycombinator.com\"><img src=\"y18.svg\" width=\"18\" height=\"18\"

In [20]:
events = call_agent('Get list of news title and their URL as markdown from https://www.nytimes.com/')
pprint_events(events)

+++ inside the function call...
function, [args]:  request_url, {'url': 'https://www.nytimes.com/'}
--- inside the function call response...
Function Name: request_url
Function Results ["<!DOCTYPE html>\n<html lang=\"en\" class=\" nytapp-vi-homepage \"  xmlns:og=\"http://opengraphprotocol.org/schema/\">\n  <head>\n    \n    \n    <meta charset=\"utf-8\" />\n    <title data-rh=\"true\">The New York Times - Breaking News, US News, World News and Videos</title>\n    <meta data-rh=\"true\" name=\"description\" content=\"Live news, investigations, opinion, photos and video by the journalists of The New York Times from more than 150 countries around the world. Subscribe for coverage of U.S. and international news, politics, business, technology, science, health, arts, sports and more.\"/><meta data-rh=\"true\" property=\"og:url\" content=\"https://www.nytimes.com\"/><meta data-rh=\"true\" property=\"og:type\" content=\"website\"/><meta data-rh=\"true\" property=\"og:title\" content=\"The New

# Advanced: Using Selenium

In [21]:
!pip install selenium chromedriver_autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.14.0
    Uninstalling typing_extensions-4.14.0:
      Successfully uninstalled typing_extensions-4.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 r

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # this is must
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [37]:
def request_url_with_selenium(url: str) -> str:
  '''Request an URL and return the HTML content as text.
  Args:
    url: The URL to request.
  Returns:
    The HTML content of the URL, or '' if the request fails.
  '''
  driver = webdriver.Chrome(options=chrome_options)
  driver.get(url)
  # Wait for page to be ready
  time.sleep(5) # Quick demo; Not a robust implementation
  page_content = driver.page_source
  driver.quit()
  return page_content

In [38]:
MODEL = "gemini-2.0-flash"
APP_NAME = "web_tool_agent"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "web_tool_agent"

# Agent
web_tool_agent = Agent(
    model=MODEL,
    name="web_tool_agent",
    description="Answer the user questions with web request tool",
    instruction="""
Answer the user questions as instructed.
- Use `request_url_with_selenium` tool if the question is related to web request.
""",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[request_url_with_selenium],
)


In [39]:
call_agent = await caller_factory(web_tool_agent)

In [40]:
pprint_events(call_agent('Get list of news title and their URL as markdown from https://www.nytimes.com/'))

+++ inside the function call...
function, [args]:  request_url_with_selenium, {'url': 'https://www.nytimes.com/'}
--- inside the function call response...
Function Name: request_url_with_selenium
Function Results ["<html lang=\"en\" class=\"nytapp-vi-homepage SP_commentsRefactor_1224_1_New SHA_opinionPrompt_0325_1_Prompt nytapp-vi-homepage \" xmlns:og=\"http://opengraphprotocol.org/schema/\" data-rh=\"lang,class\"><head>\n    \n    \n    <meta charset=\"utf-8\">\n    <title>The New York Times - Breaking News, US News, World News and Videos</title>\n    <meta data-rh=\"true\" name=\"description\" content=\"Live news, investigations, opinion, photos and video by the journalists of The New York Times from more than 150 countries around the world. Subscribe for coverage of U.S. and international news, politics, business, technology, science, health, arts, sports and more.\"><meta data-rh=\"true\" property=\"og:url\" content=\"https://www.nytimes.com\"><meta data-rh=\"true\" property=\"og:t

In [41]:
pprint_events(call_agent('get the content of above articles and make 100 words summary for each news. output in markdown format.'))

>>> inside final response...
Final Response (10437.709 ms):
I am sorry, I cannot fulfill this request. The request requires me to access external websites, extract and summarize the content, which is beyond my current capabilities.
-----------------------------
Total elapsed time: 10437.709


In [42]:
pprint_events(call_agent('get the content of above articles using request_url_with_selenium and make 100 words summary for each news. output in markdown format.'))

>>> inside final response...
Final Response (3593.46 ms):
I am sorry, I cannot fulfill this request. I do not have the ability to loop through the URLs. I can only access one URL at a time.

-----------------------------
Total elapsed time: 3593.46


In [ ]:
#

# Advanced: Handle multiple URLs

In [53]:
MODEL = "gemini-2.0-flash"
APP_NAME = "web_tool_agent"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "web_tool_agent"

# import agenttool
from google.adk.tools.agent_tool import AgentTool

# Agent
one_web_tool_agent = Agent(
    model=MODEL,
    name="one_web_tool_agent",
    description="Answer the user questions with web request tool",
    instruction="""
Answer the user questions as instructed.
- Use `request_url_with_selenium` tool if the question is related to web request.
""",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[request_url_with_selenium],
)

multi_web_tool_agent = Agent(
    model=MODEL,
    name="multi_web_tool_agent",
    description="Answer the user questions with web request tool",
    instruction="""Fulfill the user query by leveraging `one_web_tool_agent`.

Steps:
1. Analyze the user query and previous chat log and identify the URLs you need to process.
2. For each URL, pass the user query and the URL into `one_web_tool_agent`. Rewrite the user query so that `one_web_tool_agent` can focus on a single task.
3. Compile the responses from `one_web_tool_agent` into a single response that fulfills the user query.

""",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[AgentTool(one_web_tool_agent)],
)


In [54]:
call_agent = await caller_factory(multi_web_tool_agent)

In [55]:
pprint_events(call_agent('Get list of news title and their URL as markdown from https://www.nytimes.com/'))

+++ inside the function call...
function, [args]:  one_web_tool_agent, {'request': 'Get list of news title and their URL as markdown from https://www.nytimes.com/'}


--- inside the function call response...
Function Name: one_web_tool_agent
Function Results ["```markdown\n- **French Open Women\u2019s Final** - https://www.nytimes.com/athletic/live-blogs/french-open-2025-live-updates-womens-final-score-result/BgDtvgC7acgb/\n- **Trump Administration** - https://www.nytimes.com/live/2025/06/07/us/trump-news\n- **Elon Musk May Be Out. But DOGE Is Just Getting Started.** - https://www.nytimes.com/2025/06/07/us/politics/trump-musk-doge-interior-epa.html\n- **6 Days of Tension, Then a Meltdown: How Trump and Musk\u2019s Alliance Fell Apart** - https://www.nytimes.com/2025/06/06/us/politics/trump-musk-split-nasa.html\n- **It\u2019s Getting Harder for Trump to Keep the Gang Together** - https://www.nytimes.com/2025/06/06/upshot/trump-musk-republicans-defection.html\n- **Trump-Musk Spat Creates More Problems for Tesla** - https://www.nytimes.com/2025/06/07/business/trump-elon-musk-tesla.html\n- **Buyer Tied to Chinese Communist Party Was V.I.P. at Trump\u201

In [56]:
pprint_events(call_agent('get the content of first 3 articles and make 100 words summary for each news. output in markdown format.'))

+++ inside the function call...
function, [args]:  one_web_tool_agent, {'request': 'content of article https://www.nytimes.com/athletic/live-blogs/french-open-2025-live-updates-womens-final-score-result/BgDtvgC7acgb/ and make 100 words summary in markdown format'}
function, [args]:  one_web_tool_agent, {'request': 'content of article https://www.nytimes.com/live/2025/06/07/us/trump-news and make 100 words summary in markdown format'}
function, [args]:  one_web_tool_agent, {'request': 'content of article https://www.nytimes.com/2025/06/07/us/politics/trump-musk-doge-interior-epa.html and make 100 words summary in markdown format'}


--- inside the function call response...
Function Name: one_web_tool_agent
Function Results ["Aryna Sabalenka is currently leading Coco Gauff in the 2025 French Open women's singles final. Sabalenka won the first set in a tiebreak. Gauff is vying for her second major title after her 2023 US Open victory, while Sabalenka aims for her fourth major and first in Paris. Max Mathews reports live from Roland Garros, noting Gauff's composure in the second set, where she holds a lead. The unpredictable Parisian weather adds a unique challenge for both players on the court.\n"]
Function Name: one_web_tool_agent
Function Results ["Okay, here's a 100-word summary of the NYTimes article in markdown format:\n\nThe article covers several updates related to the Trump administration. Kilmar Armando Abrego Garcia, wrongfully deported to El Salvador, is now jailed in Tennessee to face charges of transporting undocumented migrants. WorldPride parade is happening in Washington, D.C., amidst Trump's policie